In [1]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [4]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [5]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama

llm_orca_mini = Ollama(
    model="orca-mini", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

### Load data

In [6]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("/Users/yinchangli/guardian/guardian/data/")

docs = loader.load()

docs_list = [doc.page_content for doc in docs]

### Split data

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

### Embedings

In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

embedding=OpenAIEmbeddings()
# model_name = 'sentence-transformers/all-MiniLM-L6-v2'
# model_name = 'intfloat/e5-large-v2'
embedding_sentence_trans = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
embedding_e5_large_v2 = HuggingFaceBgeEmbeddings(model_name = 'intfloat/e5-large-v2')

### Elastisearch

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore

In [10]:
es_db = ElasticsearchStore.from_documents(
    docs, 
    embedding, 
    es_url="http://localhost:9200", 
    index_name="index_oai",
    distance_strategy="COSINE",
)


In [11]:
es_db_e5 = ElasticsearchStore.from_documents(
    docs, 
    embedding_e5_large_v2, 
    es_url="http://localhost:9200", 
    index_name="index_e5",
    distance_strategy="COSINE",
)


In [12]:
es_db_stn_trns = ElasticsearchStore.from_documents(
    docs, 
    embedding_sentence_trans, 
    es_url="http://localhost:9200", 
    index_name="inext_stn_trns",
    distance_strategy="COSINE",
)

In [ ]:
# es_db.client.indices.refresh(index="test")

# query = "How long does it take VA to make a decision?"
# results = es_db.similarity_search(query)
# print(results)

In [13]:
es_retriever = es_db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

### Chroma Retriever

In [14]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)
chroma_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

### FAISS Retriever

In [15]:
from langchain.vectorstores import FAISS

In [16]:
faiss_vectorstore = FAISS.from_documents(docs, embedding)

In [17]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

### Self-querying

In [18]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="disability_type",
        description="The type of disability being referenced. Examples include 'PTSD', 'hearing loss', 'musculoskeletal', 'mental health', etc.",
        type="string",
    ),
    AttributeInfo(
        name="compensation_criteria",
        description="Key criteria or conditions for compensation eligibility related to the disability",
        type="string",
    ),
    AttributeInfo(
        name="benefit_rate",
        description="The rate or range of VA disability compensation for the specific disability, often dependent on severity and other factors",
        type="string",
    ),
    AttributeInfo(
        name="documentation_required",
        description="Description of the documentation required for the compensation claim, such as medical records, service records, etc.",
        type="string",
    ),
]
document_content_description = "Answers to common questions or explanations related to VA disability compensation"


In [19]:
self_retriever_chrome = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

self_retriever_es = SelfQueryRetriever.from_llm(
    llm,
    es_db,
    document_content_description,
    metadata_field_info,
)


### Ensemble Retriever

In [20]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
bm25_retriever = BM25Retriever.from_texts(docs_list)
bm25_retriever.k = 2

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

### LLM Inference

In [21]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [22]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryMemory

In [23]:
def get_response(llm, retriever,query):
    qa = ConversationalRetrievalChain.from_llm(
        llm,
        retriever,
        return_source_documents=True
    )
    result = qa({"question": query, "chat_history":[]})
    return result['answer'], result['source_documents']
    

In [24]:
# query = "what are the factors for determing the time it will take my claim?"
# query = "How long does it take VA to make a decision?"
query = 'What should I do if I disagree with my VA disability claim decision?'

In [25]:
get_response(llm, chroma_retriever, query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


("If you disagree with a VA disability claim decision that you received on or after February 19, 2019, you can ask for a review of the decision. You have three decision review options to choose from: Supplemental Claim, Higher-Level Review, or Board Appeal. If you aren't satisfied with the results of the first option you choose, you can try another eligible option.",
 [Document(page_content='11/19/23, 9:12 AM The VA Claim Process After You File Your Claim | Veterans Affairs\nhttps://www.va.gov/disability/after-you-ﬁle-claim/ 3/4What should I do if I disagree with\nmy VA disability claim decision?\nIf you disagree with a claim decision that you received on or a\x00er February 19,\n2019, you can ask us to review the decision. You have 3 decision review options\nto choose from.\xa0\nLearn about your decision review options (https://www.va.gov/decision-reviews)\nMore information about disability\nratings and payments\nEffective date of disability\n(https://www.va.gov/disability/effective-d

In [26]:
get_response(llm_orca_mini, self_retriever_chrome, query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 If you disagree with your VA disability claim decision, you should follow these steps:
1. Contact the VA and explain your concerns about your decision.
2. Ask for an explanation of how the decision was made.
3. Provide any new evidence you may have that supports your case.
4. If your claim is still disputed, request a decision review.
5. Consider appealing the decision if you are not satisfied with the outcome.
It's important to keep in mind that VA disability claims are processed in a specific order and you can only appeal one decision at a time.

(" If you disagree with your VA disability claim decision, you should follow these steps:\n1. Contact the VA and explain your concerns about your decision.\n2. Ask for an explanation of how the decision was made.\n3. Provide any new evidence you may have that supports your case.\n4. If your claim is still disputed, request a decision review.\n5. Consider appealing the decision if you are not satisfied with the outcome.\nIt's important to keep in mind that VA disability claims are processed in a specific order and you can only appeal one decision at a time.",
 [Document(page_content='11/19/23, 9:12 AM The VA Claim Process After You File Your Claim | Veterans Affairs\nhttps://www.va.gov/disability/after-you-ﬁle-claim/ 3/4What should I do if I disagree with\nmy VA disability claim decision?\nIf you disagree with a claim decision that you received on or a\x00er February 19,\n2019, you can ask us to review the decision. You have 3 decision review options\nto choose from.\xa0\nLearn about yo

In [28]:
get_response(llm, faiss_retriever, query)

("If you disagree with a VA disability claim decision that you received on or after February 19, 2019, you can ask the Veterans Affairs to review the decision. You have three decision review options to choose from: Supplemental Claim, Higher-Level Review, or Board Appeal. If you aren't satisfied with the results of the first option you choose, you can try another eligible option.",
 [Document(page_content='11/19/23, 9:12 AM The VA Claim Process After You File Your Claim | Veterans Affairs\nhttps://www.va.gov/disability/after-you-ﬁle-claim/ 3/4What should I do if I disagree with\nmy VA disability claim decision?\nIf you disagree with a claim decision that you received on or a\x00er February 19,\n2019, you can ask us to review the decision. You have 3 decision review options\nto choose from.\xa0\nLearn about your decision review options (https://www.va.gov/decision-reviews)\nMore information about disability\nratings and payments\nEffective date of disability\n(https://www.va.gov/disabil

In [29]:
get_response(llm, es_retriever, query)

('If you disagree with a VA disability claim decision that you received on or after February 19, 2019, you can ask the Veterans Affairs to review the decision. You have three decision review options to choose from.',
 [Document(page_content='11/19/23, 9:12 AM The VA Claim Process After You File Your Claim | Veterans Affairs\nhttps://www.va.gov/disability/after-you-ﬁle-claim/ 3/4What should I do if I disagree with\nmy VA disability claim decision?\nIf you disagree with a claim decision that you received on or a\x00er February 19,\n2019, you can ask us to review the decision. You have 3 decision review options\nto choose from.\xa0\nLearn about your decision review options (https://www.va.gov/decision-reviews)\nMore information about disability\nratings and payments\nEffective date of disability\n(https://www.va.gov/disability/effective-date)\nFind out when you’ll start getting your disability payments.\nWhat to expect after you get a disability rating\n(https://www.va.gov/disability/abou

### Guardrails

In [30]:
from nemoguardrails import LLMRails, RailsConfig

In [31]:
# !pwd
CONFIG_PATH = "/Users/yinchangli/guardian/guardian/Topical_Rail/"

In [32]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
config = RailsConfig.from_path(CONFIG_PATH)

In [33]:
def get_guardrail_response(llm, retriever,query):
    qa = ConversationalRetrievalChain.from_llm(
        llm,
        retriever,
        return_source_documents=True
    )
    app = LLMRails(config = config, llm = llm)
    app.register_action(qa, name = 'conversation')
    history = [
    {"role": "user", "content":query}
]
    result = app.generate(messages = history)
    return result

In [34]:
query = "How long does it take VA to make a decision?"
get_guardrail_response(llm, chroma_retriever, query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'role': 'assistant',
 'content': 'The VA typically takes 3 to 4 months to issue a decision. However, the exact time can vary depending on the complexity of the case, the amount of evidence, and the type of claim filed.'}

In [35]:
get_guardrail_response(llm_orca_mini, chroma_retriever, query)

/Users/yinchangli/env/lib/python3.10/site-packages/langchain/llms/ollama.py:186: RuntimeWarning: coroutine 'AsyncCallbackManagerForLLMRun.on_llm_new_token' was never awaited
  run_manager.on_llm_new_token(
/Users/yinchangli/env/lib/python3.10/site-packages/langchain/llms/ollama.py:186: RuntimeWarning: coroutine 'AsyncCallbackManagerForLLMRun.on_llm_new_token' was never awaited
  run_manager.on_llm_new_token(
/Users/yinchangli/env/lib/python3.10/site-packages/langchain/llms/ollama.py:186: RuntimeWarning: coroutine 'AsyncCallbackManagerForLLMRun.on_llm_new_token' was never awaited
  run_manager.on_llm_new_token(


{'role': 'assistant',
 'content': 'Is there anything else I can help you with today?'}

## RAG System Evaluation

In [36]:
import pandas as pd

df_test = pd.read_csv('questions_w_groundtruth.csv')

In [37]:
answers = []
contexts = []
for question in df_test['question']:
    answer, context = get_response(llm,self_retriever_es, question)
    print('question: ' + question)
    print('answer: ' + answer)
    print('-----------------------------')
    answers.append(answer)
    contexts.append(context)

# Assign the results back to the DataFrame
df_test['answer'] = answers
df_test['contexts'] = contexts

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


question: How long does it take VA to make a decision?
answer: The time it takes for the VA to make a decision on a claim depends on several factors including the type of claim filed, the number and complexity of injuries or disabilities claimed, and how long it takes to collect the necessary evidence to decide the claim. As of September 2023, the average number of days to complete disability-related claims was 103.3 days.
-----------------------------
question: what are the factors for determing the time it will take my claim?
answer: The time it takes to review your claim depends on these factors:

1. The type of claim you filed
2. How many injuries or disabilities you claimed and how complex they are
3. How long it takes to collect the evidence needed to decide your claim
-----------------------------
question: What should I do if I disagree with my VA disability claim decision?
answer: If you disagree with a VA disability claim decision that you received on or after February 19, 20

In [38]:
df_test

,question,ground_truths,answer,contexts
0,How long does it take VA to make a decision?,"103.3 days, the average number of days to com...",The time it takes for the VA to make a decisio...,"[page_content='11/19/23, 9:12 AM The VA Claim ..."
1,what are the factors for determing the time it...,The type of claim you filed.\nHow many injurie...,The time it takes to review your claim depends...,"[page_content='11/19/23, 9:12 AM The VA Claim ..."
2,What should I do if I disagree with my VA disa...,If you disagree with a claim decision that you...,If you disagree with a VA disability claim dec...,"[page_content='11/19/23, 9:12 AM The VA Claim ..."
3,How can I change my VA direct deposit informat...,"If you receive disability compensation, pensio...",To change your VA direct deposit information o...,"[page_content='11/19/23, 9:13 AM Change Your V..."
4,"What if I don’t have a bank account, but I wan...",The Veterans Benefits Banking Program (VBBP) p...,If you don't have a bank account but want to u...,"[page_content='11/19/23, 9:13 AM Change Your V..."
5,What evidence does VA need to support my disab...,We need evidence that shows that both of these...,The VA needs evidence that shows both of the f...,"[page_content='11/19/23, 9:18 AM Evidence Need..."
6,What happens after I file my VA disability claim?,You don’t need to do anything while you’re wai...,"After you file your VA disability claim, the f...","[page_content='11/19/23, 9:12 AM The VA Claim ..."
7,Can I submit a fully developed disability claim?,You can submit a fully developed disability cl...,"Yes, you can submit a fully developed disabili...","[page_content='11/19/23, 9:17 AM VA Fully Deve..."
8,Are there any risks to using the Fully Develop...,No. Filing a fully developed claim won’t affec...,"No, there are no risks to using the Fully Deve...","[page_content='11/19/23, 9:17 AM VA Fully Deve..."
9,Should I work with a Veterans Service Officer ...,"You don’t have to, but we encourage you to wor...",Working with a Veterans Service Officer (VSO) ...,"[page_content='11/19/23, 9:17 AM File for disa..."


#### create evaluation dataset

In [39]:
from datasets import Dataset, Features, Value, Sequence

In [40]:
df = df_test

In [41]:
data_dict = df.to_dict(orient = 'list')
df['ground_truths'] = df['ground_truths'].apply(lambda x: [x] if isinstance(x, str) else x)

In [42]:
features = Features({
    'question': Value('string'),
    'ground_truths': Value('string'),
    'answer': Value('string'),
    'contexts': Sequence(Value('string'))
})

dataset = Dataset.from_dict(data_dict, features = features)

In [43]:
dataset

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 10
})

In [44]:
llm_eval = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [45]:
embedding=OpenAIEmbeddings()

In [46]:
from ragas.metrics import AnswerCorrectness
from ragas.metrics import AnswerRelevancy
from ragas.metrics import ContextPrecision
from ragas.metrics import ContextRecall

answer_correctness = AnswerCorrectness()
answer_relevancy = AnswerRelevancy(
    embeddings=embedding
)

context_precision = ContextPrecision()
context_recall = ContextRecall()


In [47]:
ans_correctness_score = answer_correctness.score(dataset)
df['answer_correctness'] = ans_correctness_score['answer_correctness']

  0%|                                           | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████| 1/1 [00:19<00:00, 19.70s/it]


In [48]:
answer_relevancy_scores = answer_relevancy.score(dataset)
df['answer_relevancy'] = answer_relevancy_scores['answer_relevancy']

  0%|                                           | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████| 1/1 [00:07<00:00,  7.00s/it]


In [49]:
ContextPrecision_scores = context_precision.score(dataset)
df['context_precision'] = ContextPrecision_scores['context_precision']

  0%|                                           | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████| 1/1 [00:00<00:00,  1.00it/s]


In [50]:
context_recall_scores = context_recall.score(dataset)
df['context_recall'] = context_recall_scores['context_recall']

  0%|                                           | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|███████████████████████████████████| 1/1 [00:03<00:00,  3.56s/it]


In [51]:
df.to_csv('eval_result_openai.csv')